# Tarea 2 - NLP CIMAT: Minería de Texto Básica
## Por: Gustavo Hernández Angeles

## 1. Preparación (Lectura de corpus, tokenización)

In [1]:
import numpy as np
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support
import pandas as pd

### Leyendo corpus
* Utilizaré parte del código visto en la práctica.

In [2]:
def get_texts_from_file(path_corpus, path_labels):
    tr_txt = []
    tr_y = []

    with open(path_corpus, "r", encoding="utf-8-sig") as f_corpus, open(path_labels, "r", encoding="utf-8-sig") as f_truth:
        for twitt in f_corpus:
            tr_txt += [twitt.strip()]
        for label in f_truth:
            tr_y += [label.strip()]

    return tr_txt, tr_y

In [3]:
tr_txt, tr_y = get_texts_from_file("./data/mex_train.txt", "./data/mex_train_labels.txt")

In [4]:
val_txt, val_y = get_texts_from_file("./data/mex_val.txt", "./data/mex_val_labels.txt")

* Tokenizacion de Corpus  
Haré que solo lea las palabras y quitaré los stopwords

In [5]:
tokenizer = RegexpTokenizer(r'[a-zA-ZáéíóúñÁÉÍÓÚÑ]+')

In [6]:
stopwords_espanol = stopwords.words("spanish")
corpus_palabras = []
for doc in tr_txt:
    text = tokenizer.tokenize(doc)
    corpus_palabras += [w.lower() for w in text if w.lower() not in stopwords_espanol]

fdist = nltk.FreqDist(corpus_palabras)
print(f"Tamaño del vocabulario: {len(fdist)}")

Tamaño del vocabulario: 12161


* Se hace un filtrado por frecuencia, al hacer esto, reducimos el tamaño del vocabulario de 14268 a 4683  
Se escoge un umbral de 1, valor mayor reduce más el vocabulario.  
Con 2 -> el tamaño de V es 2749  
Con 3 -> el tamaño de V es 1973

In [7]:
K_umbral = 1
V = [(fdist[key], key) for key in fdist if fdist[key] > K_umbral] # Más de una ocurrencia
V.sort()
V.reverse()
print(f"Tamaño del vocabulario: {len(V)}")

Tamaño del vocabulario: 4281


In [8]:
dict_indices = dict()
cont = 0
for freq, word in V:
    dict_indices[word] = cont
    cont += 1

## 2. Bolsas de Palabras, Bigramas y Emociones

### 2.1. Evalúe BOW con pesado binario.

* Definimos una función para realizar el BOW con peso binario.

In [9]:
def binario_bow(tr_txt, V, dict_indices):
    BOW = np.zeros(shape=(len(tr_txt), len(V)), dtype=int)
    
    cont_doc = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        
        for word in fdist_doc:
            # Se elige ignorar si un término no está en el vocab.
            if word not in dict_indices:
                continue
            BOW[cont_doc, dict_indices[word]] = 1
            
        cont_doc += 1
    
    return BOW

* Definimos una función para evaluar el SVM dados las BOW de train y test.

In [10]:
def evaluar_bow(BOW_train, BOW_test, val_y):
    # Parámetro de complejidad del SVM, se proponen estos
    # y se recorrerán con GridSearch
    parameters = {"C": [.05, .12, .25, .5, 1, 2, 4]}
    # Tratar de penalizar con base a la proporción de ejemplos
    # en cada clase
    svr = svm.LinearSVC(class_weight='balanced', max_iter=10000)
    grid = GridSearchCV(estimator=svr, param_grid=parameters,
                        n_jobs=6, scoring="f1_macro", cv=5)
    grid.fit(BOW_train, tr_y)
    y_pred = grid.predict(BOW_test)

    print(confusion_matrix(val_y, y_pred))
    print(metrics.classification_report(val_y, y_pred))
    print(f"F1-score: {f1_score(val_y, y_pred, pos_label='1'):.4f}")
    
    return grid

#### Evaluamos con pesado binario

In [11]:
BOW_train = binario_bow(tr_txt, V, dict_indices)
BOW_test = binario_bow(val_txt, V, dict_indices)
svm_binario = evaluar_bow(BOW_train,BOW_test, val_y)

[[363  55]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       418
           1       0.68      0.70      0.69       169

    accuracy                           0.82       587
   macro avg       0.78      0.79      0.78       587
weighted avg       0.82      0.82      0.82       587

F1-score: 0.6939


### 2.2. Evalúe BOW con pesado frecuencia

In [12]:
# Definimos BOW con esquema de peso de frecuencia.
def frecuencia_bow(tr_txt, V, dict_indices):
    BOW = np.zeros(shape=(len(tr_txt), len(V)), dtype=int)
    
    cont_doc = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        
        for word, freq in fdist_doc.items():
            if word in dict_indices:
                BOW[cont_doc, dict_indices[word]] = freq
        cont_doc += 1
    return BOW

#### Evaluamos con pesado frecuencia

In [13]:
BOW_train = frecuencia_bow(tr_txt, V, dict_indices)
BOW_test = frecuencia_bow(val_txt, V, dict_indices)
svm_frecuencia = evaluar_bow(BOW_train,BOW_test, val_y)

[[358  60]
 [ 51 118]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.66      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6801


### 2.3. Evalúe BOW con pesado tf-idf

In [14]:
# Definimos BOW con esquema de peso de tf-idf
def tfidf_bow(tr_txt, V, dict_indices):

    
    # Un BOW de frecuencia nos ayuda a obtener tf
    # Lo haremos logaritmico
    f_bow = frecuencia_bow(tr_txt, V, dict_indices)
    tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)
    
    # De misma forma para df
    n_docs = len(tr_txt)
    df = np.count_nonzero(f_bow, axis=0)
    idf = np.log10(n_docs / (1+df))
    
    BOW = tf*idf

    return BOW

In [15]:
BOW_train = tfidf_bow(tr_txt, V, dict_indices)
BOW_test = tfidf_bow(val_txt, V, dict_indices)
svm_tfidf = evaluar_bow(BOW_train,BOW_test, val_y)

C:\Users\Gus\AppData\Local\Temp\ipykernel_12236\143517032.py:8: RuntimeWarning: divide by zero encountered in log10
  tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)


[[351  67]
 [ 57 112]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       418
           1       0.63      0.66      0.64       169

    accuracy                           0.79       587
   macro avg       0.74      0.75      0.75       587
weighted avg       0.79      0.79      0.79       587

F1-score: 0.6437


### 2.4. Evalúe BOW con pesado binario normalizado l2

In [16]:
# Función para crear BOW con pesado binario normalizado l2 (norma euclidiana)
def binarionorm_bow(tr_txt, V, dict_indices):
    BOW = binario_bow(tr_txt, V, dict_indices)
    norma = np.linalg.norm(BOW, axis=1, keepdims=True)
    norma[norma==0] = 1 # Evitar div por 0
    return BOW / norma

In [17]:
BOW_train = binarionorm_bow(tr_txt, V, dict_indices)
BOW_test = binarionorm_bow(val_txt, V, dict_indices)
svm_binnorm = evaluar_bow(BOW_train,BOW_test, val_y)

[[355  63]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.86       418
           1       0.65      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6781


### 2.5. Evalúe BOW con pesado frecuencia normalizado l2

In [18]:
# Función BOW pesado frecuencia normalizado l2
def frecuencianorm_bow(tr_txt, V, dict_indices):
    BOW = frecuencia_bow(tr_txt, V, dict_indices)
    norma = np.linalg.norm(BOW, axis=1, keepdims=True)
    norma[norma==0] = 1
    return BOW/norma

In [19]:
BOW_train = frecuencianorm_bow(tr_txt, V, dict_indices)
BOW_test = frecuencianorm_bow(val_txt, V, dict_indices)
svm_frecnorm = evaluar_bow(BOW_train,BOW_test, val_y)

[[351  67]
 [ 47 122]]
              precision    recall  f1-score   support

           0       0.88      0.84      0.86       418
           1       0.65      0.72      0.68       169

    accuracy                           0.81       587
   macro avg       0.76      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6816


### 2.6. Evalúe BOW con pesado tfidf normalizado l2

In [20]:
def tfidfnorm_bow(tr_txt, V, dict_indices):
    BOW = tfidf_bow(tr_txt, V, dict_indices)
    norma = np.linalg.norm(BOW, axis=1, keepdims=True)
    norma[norma==0] = 1
    return BOW/norma

In [21]:
BOW_train = tfidfnorm_bow(tr_txt, V, dict_indices)
BOW_test = tfidfnorm_bow(val_txt, V, dict_indices)
svm_tfidfnorm = evaluar_bow(BOW_train,BOW_test, val_y)

C:\Users\Gus\AppData\Local\Temp\ipykernel_12236\143517032.py:8: RuntimeWarning: divide by zero encountered in log10
  tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)


[[358  60]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.66      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.78       587
weighted avg       0.82      0.81      0.81       587

F1-score: 0.6839


### 2.7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.

Representación  | Macro F1
----------------|----------
Binario         | **69.39%**
Frecuencia      | 68.01%
TF-IDF          | 64.37%
Binario L2      | 67.81%
Frecuencia L2   | 68.16%
TF-IDF L2       | 68.39%

Aún cuando el esquema de pesado TF-IDF puede tener en cuenta más aspectos de los términos, en este problema arroja un score inferior al binario, el cual es muy simple.

### 2.8. De las configuraciones anteriores eliga la mejor y evalúela con más y menos términos

#### Todo el vocabulario con al menos dos ocurrencias

In [22]:
V = [(fdist[key], key) for key in fdist if fdist[key] > 1]
V.sort()
V.reverse()
longitud_vocabulario = len(V)
print(f"Tamaño del vocabulario: {longitud_vocabulario}")

dict_indices = dict()
cont = 0
for freq, word in V:
    dict_indices[word] = cont
    cont += 1
print(f"Tamaño del diccionario: {len(dict_indices)}")

Tamaño del vocabulario: 4281
Tamaño del diccionario: 4281


In [23]:
BOW_train = binario_bow(tr_txt, V, dict_indices)
BOW_test = binario_bow(val_txt, V, dict_indices)
svm_masPalabras = evaluar_bow(BOW_train,BOW_test, val_y)

[[363  55]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       418
           1       0.68      0.70      0.69       169

    accuracy                           0.82       587
   macro avg       0.78      0.79      0.78       587
weighted avg       0.82      0.82      0.82       587

F1-score: 0.6939


#### Mitad del Vocabulario

In [24]:
V_mitad = V[:int(longitud_vocabulario/2)]
dict_indices = dict()
cont = 0
for freq, word in V_mitad:
    dict_indices[word] = cont
    cont += 1
print(f"Tamaño del vocabulario: {len(V_mitad)}")
print(f"Tamaño del diccionario: {len(dict_indices)}")

Tamaño del vocabulario: 2140
Tamaño del diccionario: 2140


In [25]:
BOW_train = binario_bow(tr_txt, V_mitad, dict_indices)
BOW_test = binario_bow(val_txt, V_mitad, dict_indices)
svm_mitadPalabras = evaluar_bow(BOW_train,BOW_test, val_y)

[[360  58]
 [ 47 122]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.68      0.72      0.70       169

    accuracy                           0.82       587
   macro avg       0.78      0.79      0.79       587
weighted avg       0.82      0.82      0.82       587

F1-score: 0.6991


#### 1000 palabras

In [26]:
V_1000 = V[:1000]
dict_indices = dict()
cont = 0
for freq, word in V_1000:
    dict_indices[word] = cont
    cont += 1
print(f"Tamaño del vocabulario: {len(V_1000)}")
print(f"Tamaño del diccionario: {len(dict_indices)}")

Tamaño del vocabulario: 1000
Tamaño del diccionario: 1000


In [27]:
BOW_train = binario_bow(tr_txt, V_mitad, dict_indices)
BOW_test = binario_bow(val_txt, V_mitad, dict_indices)
svm_1000 = evaluar_bow(BOW_train,BOW_test, val_y)

[[358  60]
 [ 51 118]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.66      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6801


#### Tabla comparativa de binaria


Términos utilizados  | Macro F1
----------------|----------
4281         | 69.39%
2140      | **69.91%**
1000          | 68.01%

Resultó mejor utiilzar la mitad de las palabras con una ventaja de +0.6%

### 2.9. Recurso Léxico EmoLex para construir una Bolsa de Emociones (BoE)

In [28]:
emolex_path = "./data/Spanish-NRC-EmoLex.txt"
emolex = pd.read_csv(emolex_path, sep="\t")

El recurso que encontré se encuentra en `./data/Spanish-NRC-EmoLex.txt`. Inspeccionandolo encuentro que:
* Cada renglón es una palabra, al que le corresponde un *vector de emociones* incluyendo el sentimiento **positivo** o **negativo**.
* Las palabras provienen del inglés y son traducidas al español, por lo que puede haber palabras repetidas.

In [29]:
emolex.head()

,English Word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Spanish Word
0,aback,0,0,0,0,0,0,0,0,0,0,detrás
1,abacus,0,0,0,0,0,0,0,0,0,1,ábaco
2,abandon,0,0,0,1,0,1,0,1,0,0,abandonar
3,abandoned,1,0,0,1,0,1,0,1,0,0,abandonado
4,abandonment,1,0,0,1,0,1,0,1,1,0,abandono


Quitaré las dimensiones `positivo`, `negativo` y `English Word`.

In [30]:
clean_emolex = emolex.drop(columns=["English Word", "positive", "negative"])
clean_emolex.head()

,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,Spanish Word
0,0,0,0,0,0,0,0,0,detrás
1,0,0,0,0,0,0,0,1,ábaco
2,0,0,0,1,0,1,0,0,abandonar
3,1,0,0,1,0,1,0,0,abandonado
4,1,0,0,1,0,1,1,0,abandono


Siguiendo la propuesta de enmascarar cada término con su emoción, pienso representar cada tweet las contribuciones de sus palabras.


#### Funciones para los diferentes esquemas de pesado

In [31]:
def crearBoE_frecuencia(tr_txt, emolex):
    BoE = np.zeros(shape=(len(tr_txt), 8))
    
    for i, tweet in enumerate(tr_txt):
        vector = np.zeros(8)
        palabras = tokenizer.tokenize(tweet.lower())
        
        # Obtener solo palabras en el lexicon
        ocurrencias = emolex[emolex["Spanish Word"].isin(palabras)]
        
        if not ocurrencias.empty:
            # De forma frecuencia
            vector = ocurrencias.drop(columns="Spanish Word").sum(axis=0).to_numpy()
    
        BoE[i] = vector
        
    return BoE


def crearBoE_binaria(tr_txt, emolex):
    # De frecuencia transformamos a binaria
    BoE = crearBoE_frecuencia(tr_txt, emolex)
    BoE = np.where(BoE > 0, 1, 0)
    return BoE


def crearBoE_tfidf(tr_txt, emolex):
    # De forma muy similar a BoW
    
    # Un BOW de frecuencia nos ayuda a obtener tf
    # Lo haremos logaritmico
    f_bow = crearBoE_frecuencia(tr_txt, emolex)
    tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)
    
    # De misma forma para df
    n_docs = len(tr_txt)
    df = np.count_nonzero(f_bow, axis=0)
    idf = np.log10(n_docs / (1+df))

    BoE = tf*idf
    
    return BoE


#### Evaluamos cada esquema de pesado

In [32]:
bag_of_emotions_train = crearBoE_binaria(tr_txt, clean_emolex)
bag_of_emotions_test = crearBoE_binaria(val_txt, clean_emolex)

svm_emolex_binario = evaluar_bow(bag_of_emotions_train, bag_of_emotions_test, val_y)

[[344  74]
 [115  54]]
              precision    recall  f1-score   support

           0       0.75      0.82      0.78       418
           1       0.42      0.32      0.36       169

    accuracy                           0.68       587
   macro avg       0.59      0.57      0.57       587
weighted avg       0.66      0.68      0.66       587

F1-score: 0.3636


In [33]:
bag_of_emotions_train = crearBoE_frecuencia(tr_txt, clean_emolex)
bag_of_emotions_test = crearBoE_frecuencia(val_txt, clean_emolex)

svm_emolex_frecuencia = evaluar_bow(bag_of_emotions_train, bag_of_emotions_test, val_y)

[[357  61]
 [125  44]]
              precision    recall  f1-score   support

           0       0.74      0.85      0.79       418
           1       0.42      0.26      0.32       169

    accuracy                           0.68       587
   macro avg       0.58      0.56      0.56       587
weighted avg       0.65      0.68      0.66       587

F1-score: 0.3212


In [34]:
bag_of_emotions_train = crearBoE_tfidf(tr_txt, clean_emolex)
bag_of_emotions_test = crearBoE_tfidf(val_txt, clean_emolex)

svm_emolex_tfidf = evaluar_bow(bag_of_emotions_train, bag_of_emotions_test, val_y)

C:\Users\Gus\AppData\Local\Temp\ipykernel_12236\768315645.py:33: RuntimeWarning: divide by zero encountered in log10
  tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)


[[346  72]
 [116  53]]
              precision    recall  f1-score   support

           0       0.75      0.83      0.79       418
           1       0.42      0.31      0.36       169

    accuracy                           0.68       587
   macro avg       0.59      0.57      0.57       587
weighted avg       0.66      0.68      0.66       587

F1-score: 0.3605


C:\Users\Gus\AppData\Local\Temp\ipykernel_12236\768315645.py:33: RuntimeWarning: divide by zero encountered in log10
  tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)


### 2.10. Tabla comparativa


Representación  | Macro F1
----------------|----------
Binario         | **36.36%**
Frecuencia      | 32.12%
TF-IDF          | 36.05%

Los resultados no son nada optimistas con mi propuesta. Si el caso fue diferente para otras propuestas me gustaría conocerlas.

## 3. Recurso Linguistico de Emociones Mexicano

### 3.1. Utilizar el recurso Léxico "Spanish Emotion Lexicon" (SEL).

El recurso léxico llamado "Spanish Emotion Lexicon (SEL)" contiene las palabras en español asociadas a una emoción básica. Estas emociones pueden ser: *Alegría*, *Enojo*, *Miedo*, *Repulsión*, *Sorpresa*, *Tristeza*. 

También contiene una métrica llamada **Probability of Affective Use (PFA)** la cual refleja la probabilidad de que la palabra se utilice en asociación con la emoción ligada.

Mi propuesta es algo similar al recurso léxico EmoLex. Representaré a los términos como un vector $\vec{v}$ de emociones que tendrá en cada componente i-esima el valor $PFA$ correspondiente a la i-esima emoción. $\vec{v_i} = {PFA}_i$

* Para pesado de frecuencia.  
    Representaría a cada documento como la suma de los vectores de cada palabra que contenga. 

    $$\vec{d}_k = \sum_{v \in d_k} \vec{v}$$

    También normalizaré para que la suma de cada componente del documento sea igual a 1.

    $$ \sum_{j} {d_k}_j = 1  $$

* Para pesado binario.  
    Utilizaré el resultado del pesado frecuencia, simplemente aplicar algo similar a la función *ceil* para cada componente.

* Para pesado tfidf.  
    Transformaré a partir de la representación tf a tfidf, similar a EmoLex.

In [35]:
sel_path = "./data/SEL.txt"
sel = pd.read_csv(sel_path, sep="\t")
sel.drop(columns=["#"], inplace=True)

In [36]:
sel.head()

,Palabra,Nula[%],Baja[%],Media[%],Alta[%],PFA,Categoría
0,abundancia,0,0,50,50,0.830,Alegría
1,acabalar,40,0,60,0,0.396,Alegría
2,acallar,50,40,10,0,0.198,Alegría
3,acatar,50,40,10,0,0.198,Alegría
4,acción,30,30,30,10,0.397,Alegría


In [37]:
print(len(sel["Categoría"].unique()))
sel["Categoría"].unique()

6


array(['Alegría', 'Enojo', 'Miedo', 'Repulsión', 'Sorpresa', 'Tristeza'],
      dtype=object)

#### Funciones para crear BoE

In [38]:
def sel_crearBoE_frecuencia(tr_txt, sel : pd.DataFrame):
    
    emociones_indice = {
        "Alegría" : 0,
        "Enojo" : 1,
        "Miedo" : 2,
        "Repulsión" : 3,
        "Sorpresa" : 4,
        "Tristeza" : 5
    }
    
    n_emociones = len(emociones_indice)
    
    BoE = np.zeros(shape=(len(tr_txt), n_emociones))
    
    for i, tweet in enumerate(tr_txt):
        vector = np.zeros(n_emociones)
        palabras = tokenizer.tokenize(tweet.lower())
        
        # Obtener solo palabras en el lexicon
        ocurrencias = sel[sel["Palabra"].isin(palabras)]
        
        if not ocurrencias.empty:
            # De forma frecuencia
            for row in ocurrencias.itertuples():
                # Vemos a qué componente refiere la emoción
                # y le sumamos su PFA.
                indice = emociones_indice[row.Categoría]
                vector[indice] += row._6
    
        # Normalizado
        BoE[i] = vector / n_emociones
        
    return BoE

def sel_crearBoE_binaria(tr_txt, sel):
    # De frecuencia transformamos a binaria
    BoE = sel_crearBoE_frecuencia(tr_txt, sel)
    BoE = np.where(BoE > 0, 1, 0)
    return BoE


def sel_crearBoE_tfidf(tr_txt, sel):
    # De forma muy similar a BoW
    
    # Un BOW de frecuencia nos ayuda a obtener tf
    # Lo haremos logaritmico
    f_bow = sel_crearBoE_frecuencia(tr_txt, sel)
    tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)
    
    # De misma forma para df
    n_docs = len(tr_txt)
    df = np.count_nonzero(f_bow, axis=0)
    idf = np.log10(n_docs / (1+df))

    BoE = tf*idf
    
    return BoE

#### Evaluamos cada esquema de pesado

In [39]:
# Binario
bag_of_emotions_train = sel_crearBoE_binaria(tr_txt, sel)
bag_of_emotions_test = sel_crearBoE_binaria(val_txt, sel)

svm_sel_binario = evaluar_bow(bag_of_emotions_train, bag_of_emotions_test, val_y)

[[ 84 334]
 [ 33 136]]
              precision    recall  f1-score   support

           0       0.72      0.20      0.31       418
           1       0.29      0.80      0.43       169

    accuracy                           0.37       587
   macro avg       0.50      0.50      0.37       587
weighted avg       0.59      0.37      0.35       587

F1-score: 0.4257


In [40]:
# Frecuencia
bag_of_emotions_train = sel_crearBoE_frecuencia(tr_txt, sel)
bag_of_emotions_test = sel_crearBoE_frecuencia(val_txt, sel)

svm_sel_frecuencia = evaluar_bow(bag_of_emotions_train, bag_of_emotions_test, val_y)

[[ 84 334]
 [ 31 138]]
              precision    recall  f1-score   support

           0       0.73      0.20      0.32       418
           1       0.29      0.82      0.43       169

    accuracy                           0.38       587
   macro avg       0.51      0.51      0.37       587
weighted avg       0.60      0.38      0.35       587

F1-score: 0.4306


In [41]:
# TFIDF
bag_of_emotions_train = sel_crearBoE_tfidf(tr_txt, sel)
bag_of_emotions_test = sel_crearBoE_tfidf(val_txt, sel)

svm_sel_tfidf = evaluar_bow(bag_of_emotions_train, bag_of_emotions_test, val_y)

C:\Users\Gus\AppData\Local\Temp\ipykernel_12236\2882340352.py:49: RuntimeWarning: divide by zero encountered in log10
  tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)


[[ 66 352]
 [ 30 139]]
              precision    recall  f1-score   support

           0       0.69      0.16      0.26       418
           1       0.28      0.82      0.42       169

    accuracy                           0.35       587
   macro avg       0.49      0.49      0.34       587
weighted avg       0.57      0.35      0.30       587

F1-score: 0.4212


#### Tabla comparativa.

Representación  | Macro F1
----------------|----------
Binario         | 42.57%
Frecuencia      | **43.06%**
TF-IDF          | 42.12%

En este caso, el esquema de pesado de frecuencia fue el que mejor puntaje obtuvo utilizando el recurso léxico SEL.

### 3.2. Sobre la estrategía para incorporar el PFA

La representación que utilicé fue pensando en que un determinado tweet solo podría tener varias emociones asociadas y cada una tiene su PFA. Significando que la probabilidad de que el tweet esté asociada a la i-esima emoción está dada por su i-esima componente. Es por esto que normalizo para que la suma de todas las componentes sea igual a 1.


## 4. ¿Podemos mejorar con Bigramas?

### 4.1. Concatenar BoW con otra BoW con 1000 bigramas más frecuentes.

In [42]:
# Para determinar los 1000 bigramas más frecuentes
# utilizaré el corpus sin stopwords.
from nltk import bigrams
from collections import Counter

lista_bigramas = list(bigrams((corpus_palabras)))

freq_bigramas = nltk.FreqDist(lista_bigramas)

# 1000 bigramas más comunes
bigramas_mas_comunes = [(w1,w2) for (w1,w2), freq in freq_bigramas.most_common(1000)]

bigramas_mas_comunes[:2]

[('usuario', 'usuario'), ('puta', 'madre')]

In [43]:
# La mejor BoW que obtuve según los experimentos
# fue con pesado binario recortando a la mitad de palabras.

V_mitad = V[:int(longitud_vocabulario/2)]
dict_indices = dict()
cont = 0

# Concatenamos
for freq, word in V_mitad:
    dict_indices[word] = cont
    cont += 1
for w1, w2 in bigramas_mas_comunes:
    dict_indices[w1+w2] = cont
    cont += 1

In [44]:
def binario_bow_bigram(tr_txt, V, bigramas, dict_indices):
    
    BOW = np.zeros(shape=(len(tr_txt), len(V)+len(bigramas)), dtype=int)
    
    cont_doc = 0
    for tr in tr_txt:
        palabras = tokenizer.tokenize(tr)
        fdist_doc = nltk.FreqDist(palabras)
        
        # Términos
        for word in fdist_doc:
            # Se elige ignorar si un término no está en el vocab.
            if word not in dict_indices:
                continue
            BOW[cont_doc, dict_indices[word]] = 1
            
        # Bigramas
        lista_bigramas = list(bigrams((palabras)))
        for b1,b2 in lista_bigramas:
            # Se elige ignorar si un bigrama no está en nuestros bigramas
            if b1+b2 not in dict_indices:
                continue
            BOW[cont_doc, dict_indices[b1+b2]] = 1
        
        cont_doc += 1
    
    return BOW

In [45]:
# evaluamos BoW+Bigramas
BOW_train = binario_bow_bigram(tr_txt, V_mitad, bigramas_mas_comunes, dict_indices)
BOW_test = binario_bow_bigram(val_txt, V_mitad, bigramas_mas_comunes, dict_indices)
svm_bow_con_bigramas = evaluar_bow(BOW_train,BOW_test, val_y)

[[358  60]
 [ 47 122]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.67      0.72      0.70       169

    accuracy                           0.82       587
   macro avg       0.78      0.79      0.78       587
weighted avg       0.82      0.82      0.82       587

F1-score: 0.6952


El rendimiento no mejoró, bajó con una diferencia de 0.4%. Sin embargo, se sigue manteniendo sobre los otros experimentos.

### 4.2. Experimento con BoE+BoW+Bigramas

In [46]:
# Hacemos todas las bolsas: emolex, sel, bow+bigramas.
# En particular, las que tuvieron mejor puntuación por su esquema de pesado.

bag_of_emotions_train = crearBoE_binaria(tr_txt, clean_emolex)
bag_of_emotions_test = crearBoE_binaria(val_txt, clean_emolex)

sel_BOW_tr = sel_crearBoE_frecuencia(tr_txt, sel)
sel_BOW_val = sel_crearBoE_frecuencia(val_txt, sel)

bow_bigramas_tr = binario_bow_bigram(tr_txt, V_mitad, bigramas_mas_comunes, dict_indices)
bow_bigramas_val = binario_bow_bigram(val_txt, V_mitad, bigramas_mas_comunes, dict_indices)

# Los unimos con np.concatenate
boe_bow_bigram_tr = np.concatenate([bag_of_emotions_train, sel_BOW_tr, bow_bigramas_tr], axis = 1)
boe_bow_bigram_val = np.concatenate([bag_of_emotions_test, sel_BOW_val, bow_bigramas_val], axis = 1)

In [47]:
# Lo evaluamos
svm_boe_bow_bigram = evaluar_bow(boe_bow_bigram_tr, boe_bow_bigram_val, val_y)

[[358  60]
 [ 49 120]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.67      0.71      0.69       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.78       587
weighted avg       0.82      0.81      0.82       587

F1-score: 0.6877


### 4.3 Conclusiones

A veces menos es más.

En este trabajo implementé varias formas de representar a los documentos; sus términos, bigramas, palabras asociadas a emociones e incluso la combinación de todas. Sin embargo, aunque parecieran modelos más complejos y que toman en cuenta más variables, el desempeño del modelo **no mejoraba**. La mejor puntuación fue con una **BoW simple de esquema binario** y con un vocabulario recortado a la mitad. 

El costo computacional también fue incrementando conforme a los experimentos. Por ejemplo, en este último experimento donde combinamos todas las Bolsas tardó alrededor de 9 segundos en crear las bolsas de training y validación, mientras que el BoW simple de esquema binario tardó menos de 2 segundos en crearlo y entrenar el SVM. Esta diferencia me hace pensar que muchas veces las ideas más complicadas y con mayores variables pueden dar resultados **iguales o hasta peores**, al menos cuando hablamos de clasificar documentos. No creo que hayan ayudado mucho en este caso.